In [1]:
List_datasets = ['Jan and Feb.xlsx', 'March and April.xlsx',  'May June.xlsx',
                 'July and August .xlsx', 'Sep oct.xlsx', 'Nov.xlsx',
                'Dec.xlsx']

# Importing Data And Libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from geopy.geocoders import Nominatim
import time
from pprint import pprint

In [4]:
latitude = []
longitude = []

def get_location_by_address(address):
    """This function returns a location as raw from an address
    will repeat until success"""
    time.sleep(1)
    try:
        return app.geocode(address).raw
    except:
        pass

In [5]:
df_o  = pd.DataFrame()

for dataframe in List_datasets:
    dataset = pd.read_excel(dataframe)
    df_o = pd.concat([df_o, dataset])

df = df_o.copy()

In [6]:
root_path = 'Desktop/'
os.mkdir(os.path.join(root_path, 'مبيعات المناطق'))
os.chdir(os.path.join(root_path, 'مبيعات المناطق'))

# Cleaning Part

### Extract the year and the month columns


In [7]:
df['day'] = pd.to_datetime(df['تاريخ الاضافة']).dt.day
d = list(df['day'].dropna().unique())

df['month'] = pd.to_datetime(df['تاريخ الاضافة']).dt.month_name()
m = list(df['month'].dropna().unique())

df['year'] = pd.to_datetime(df['تاريخ الاضافة']).dt.year
y = list(df['year'].dropna().unique())

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_15936\3301682872.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['day'] = pd.to_datetime(df['تاريخ الاضافة']).dt.day
C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_15936\3301682872.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['month'] = pd.to_datetime(df['تاريخ الاضافة']).dt.month_name()
C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_15936\3301682872.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['year'] = pd.to_datetime(df['تاريخ الاضافة']).dt.year


In [8]:
list_of_cities= list(df['المنطقة الجغرافية'].unique())
cities_df = pd.DataFrame()
cities_df['المنطقة الجغرافية'] = list_of_cities

In [9]:
latitude = []
longitude = []
app = Nominatim(user_agent="tutorial")

for city in list_of_cities:
    try:
        location=  app.geocode(' السعودية ' + city).raw
        latitude.append(location["lat"])
        longitude.append(location["lon"])
    except:
        latitude.append(0)
        longitude.append(0)


In [10]:
cities_df['lat'] = latitude
cities_df['lon'] = longitude
df = cities_df.merge(df, on = 'المنطقة الجغرافية', how = 'inner')

In [11]:
invoices= pd.pivot_table(df,index = ['المنطقة الجغرافية'], values=(['رقم الفاتورة']), aggfunc = 'nunique')
invoices.rename(columns ={'رقم الفاتورة':'عدد الفواتير'}, inplace = True)

Product_sales= pd.pivot_table(df,index = ['المنطقة الجغرافية'], values=(['الاجمالى.1']), aggfunc = 'sum')
Product_sales.rename(columns ={'الاجمالى.1':'اجمالي المبيعات'}, inplace = True)

quantity= pd.pivot_table(df,index = ['المنطقة الجغرافية'], values=(['الكمية']), aggfunc = 'sum')
quantity.rename(columns ={'الكمية':'الوحدات المباعة من المنتجات'}, inplace = True)

ship= pd.pivot_table(df,index = ['المنطقة الجغرافية'], values=(['تكلفة الشحن']), aggfunc = 'mean')
ship.rename(columns ={'تكلفة الشحن':'متوسط تكلفة الشحن'}, inplace = True)

df_sales =  invoices.copy()
df_sales =  df_sales.merge(Product_sales, on = 'المنطقة الجغرافية', how = 'left')
df_sales =  df_sales.merge(quantity, on = 'المنطقة الجغرافية', how = 'left')
df_sales =  df_sales.merge(ship, on = 'المنطقة الجغرافية', how = 'left')

df_sales.sort_values(by= "اجمالي المبيعات", ascending = False).to_excel('sales report.xlsx', sheet_name='مبيعات المدن')

In [12]:
df[['المنطقة الجغرافية', 'lat', 'lon']]
df_sales.reset_index(inplace = True)
df_sales = df_sales.merge(df[['المنطقة الجغرافية', 'lat', 'lon']].drop_duplicates(), on = 'المنطقة الجغرافية', how = 'inner')
df_sales.set_index('المنطقة الجغرافية')
df_sales['متوسط تكلفة الشحن نسبة'] =(df_sales['متوسط تكلفة الشحن'] - df_sales['متوسط تكلفة الشحن'].min()) / (df_sales['متوسط تكلفة الشحن'].max() - df_sales['متوسط تكلفة الشحن'].min())

In [13]:
df_sales.to_excel('power_BI.xlsx')

In [14]:
data_sales = pd.DataFrame()
regions = list(df['المنطقة الجغرافية'].unique())
for x in regions: 
    try:
        region = df[df['المنطقة الجغرافية'] == x]
        Total_sold = pd.pivot_table(region, index=['المنطقة الجغرافية', 'اسم المنتج'], values=(['الكمية']), aggfunc = 'sum')
        Total_sold.rename(columns ={'الكمية':'sold units'}, inplace = True)
        Total_sold = Total_sold.sort_values(by= 'sold units', ascending = False).head(5)
    except:
        continue  
    total_sales = pd.pivot_table(region, index=['المنطقة الجغرافية', 'اسم المنتج'], values=(['الاجمالى.1']), aggfunc = 'sum')
    total_sales.rename(columns ={ 'الاجمالى.1':'اجمالي المبيعات'}, inplace = True)
    df2 = Total_sold.merge(total_sales, on= ['المنطقة الجغرافية', 'اسم المنتج'], how= 'inner')
    df2.sort_values(by= 'اجمالي المبيعات', ascending = False)
    
    h = len(df2) + 1
    df2.loc[' ', df2.columns] = ' '
    df2.loc['  ', df2.columns] = ' '
    df2.loc[('المنطقة الجغرافية', 'اسم المنتج'), df2.columns] = df2.columns
    data_sales  = pd.concat([df2, data_sales], ignore_index=False)
    data_sales.merge

data_sales.to_excel(' products Per region.xlsx', sheet_name= 'Sales')   

In [15]:
data_sales = pd.DataFrame()
for x in regions: 
        region = df[df['المنطقة الجغرافية'] == x]
        pay_op = pd.pivot_table(region,index = ['المنطقة الجغرافية', 'طريقة الدفع'], values=(['رقم الفاتورة']), aggfunc = 'count')
        pay_op.rename(columns ={'رقم الفاتورة':  'عدد مرات الاستخدام'}, inplace = True)
        pay_op = pay_op.sort_values(by= 'عدد مرات الاستخدام', ascending = False).head(3)
        h = len(pay_op) + 1
        pay_op.loc[' ', pay_op.columns] = ' '
        pay_op.loc['  ', pay_op.columns] = ' '
        pay_op.loc[('المنطقة الجغرافية', 'طريقة الدفع'), pay_op.columns] = pay_op.columns
        data_sales  = pd.concat([pay_op, data_sales], ignore_index=False)
        
        
with pd.ExcelWriter((' products Per region.xlsx'), engine='openpyxl', mode='a') as writer:
    data_sales.to_excel(writer, sheet_name= 'طرق الدفع')          
